In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=N7801icA81zUqiK7528WRVA86aPVRnGd9sMy1IsVhFM&tc=DmwIsHoWpg31XNCHH4n7upu8cFob84BiY5mIkgjwVEI&cc=yjkDXXxbIHkrPngC9r0smuUsU3Lkt4NzriCRgBT0gio

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmYSjfTOj2oXtyJ4mxK2QIt4u-mFsQ66Fc8EC8QCmfPe6_VNyZ1yvY

Successfully saved authorization token.


In [3]:
FL = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('STUSPS', 'FL'))

#USGS Landsat 7 Collection 2 Tier 1 Raw Scenes for summer 2000
collection = ee.ImageCollection('LE7_L1T').filterDate('2000-06-01', '2000-10-31').filterBounds(FL)

image1 = collection.median()
image2 = image1.clipToCollection(FL)

In [4]:
#Selecting red, green, blue bands
image = image2.select('B4','B3','B2')

In [5]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[27.994402, -81.760254], zoom_start=6)

# Add the layer to the map object.
my_map.add_ee_layer(image, {'gain': [1.4, 1.4, 1.1]}, 'Landsat 7')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)